In [ ]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback


model = TQC.load("data/tqc1/tqc_model_873000_steps")
env = roboverse.make("Widow250PickPlace-v2",
                         gui=True,
                         observation_mode="pixels",
                         transpose_image=False)
model.set_env(env)
env = model.get_env()

obs = env.reset()
print("start render")
for i in range(int(1e4)):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print(rewards)
    env.render("human")
env.close()

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [1]:
import time
import numpy as np
from roboverse.policies import policies
from stable_baselines3.common.utils import set_random_seed


def make_env(env_id: str, rank: int, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = roboverse.make(env_id,
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
        #env = TimeFeatureWrapper(env)
        #env.reset(seed=seed + rank)
        env.reset()
        return env
    set_random_seed(seed)
    return _init


def collect_data(env, model, policy, target, num_trajectories=100, num_timesteps=30):
    policy_class = policies[policy]
    policy = policy_class(env)
    num_success = 0
    num_saved = 0
    accept_trajectory_key = target
    noise = 0.1
    EPSILON = 0.1

    while num_saved < num_trajectories:
        num_saved += 1
        num_steps = 1e6
        rewards = []
        env.reset()
        policy.reset()
        time.sleep(0.1)
        success = False
        for j in range(num_timesteps):
            action, agent_info = policy.get_action()

            # In case we need to pad actions by 1 for easier realNVP modelling 
            env_action_dim = env.action_space.shape[0]
            #if env_action_dim - action.shape[0] == 1:
            #    action = np.append(action, 0)
            action += np.random.normal(scale=noise, size=(env_action_dim,))
            action = np.clip(action, -1 + EPSILON, 1 - EPSILON)
            observation = env.get_observation()
            observation["image"] = np.transpose(observation["image"], (2, 0, 1))
            next_observation, reward, done, info = env.step(action)
            next_observation["image"] = np.transpose(next_observation["image"], (2, 0, 1))
            rewards.append(reward)
            success = sum(rewards) > 70
            model.replay_buffer.add(observation, next_observation, action, reward, np.array([done]), [{}])

            if success and num_steps > 1e3: #info[accept_trajectory_key]
                num_steps = j

            if success and j > 23: #info[accept_trajectory_key]
                break
            if done or agent_info['done']:
                break

        if success: #info[accept_trajectory_key]
            PRINT = False
            if PRINT:
                print("num_timesteps: ", num_steps, rewards)
                #print(observation["image"].shape)
                #print(next_observation["image"].shape)
            num_success += 1
        if num_saved%100 == 0:
            print(f"num_trajectories: {num_saved} success rate: {num_success/num_saved} Reward: {sum(rewards)}")

    print("success rate: {}".format(num_success / (num_saved)))


pybullet build time: May 20 2022 19:44:17


In [6]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


env = roboverse.make("Widow250PickPlace-v2",
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
#env = TimeFeatureWrapper(env)
#env = DummyVecEnv([make_env("Widow250PickPlace-v1", i) for i in range(4)])
seed = 1
obs = env.reset()

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path=f"./data/seed_{seed}/",
  name_prefix="tqc_model",
  save_replay_buffer=False,
  save_vecnormalize=False,
)

model = TQC(env=env, batch_size=2048, buffer_size=1_000_000, gamma=0.95, learning_rate=0.001, policy='MultiInputPolicy',
             policy_kwargs=dict(net_arch=[512, 512, 512], n_critics=2),
             replay_buffer_class=HerReplayBuffer,
             replay_buffer_kwargs=dict(goal_selection_strategy='future', n_sampled_goal=4),
             tau=0.05, learning_starts=200, verbose=1)

#model = TQC.load("data/tqc")
#model.set_env(env)
COLLECT=False
if COLLECT:
    collect_data(env, model, "pickplace", "place_success_target", 10000, 35)
    model.save_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")
else:
    print("load_replay_buffer")
    model.load_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")

print("start pre-training from buffer only")
model.learn(total_timesteps=0, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.train(gradient_steps=20000)

print("start learning")
model.learn(total_timesteps=480_000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save(f"data/seed_{seed}/tqc_pick_place")
model.save_replay_buffer(f"data/seed_{seed}/tqc_trained_pick_place")

print("load_replay_buffer")
model.load_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")
collect_data(env, model, "pickplace", "place_success_target", 10000, 35)
model.save_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")

model.learn(total_timesteps=500_000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save(f"data/seed_{seed}/tqc_pick_place")
model.save_replay_buffer(f"data/seed_{seed}/tqc_trained_pick_place")

print("finish learning")

/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment Widow250PickPlace-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional informat

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
load_replay_buffer


Output()

start pre-training from buffer only


/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/stable_baselines3/her/her_replay_buffer.py:394: UserWarning: The last trajectory in the replay buffer will be truncated.
If you are in the same episode as when the replay buffer was saved,
you should use `truncate_last_trajectory=False` to avoid that issue.
  warnings.warn(


...

start learning
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 99        |
|    ep_rew_mean     | -99       |
| time/              |           |
|    episodes        | 5         |
|    fps             | 20        |
|    time_elapsed    | 24        |
|    total_timesteps | 495       |
| train/             |           |
|    actor_loss      | -2.62e+04 |
|    critic_loss     | 3.15e+04  |
|    ent_coef        | 10.7      |
|    ent_coef_loss   | 2.04      |
|    learning_rate   | 0.001     |
|    n_updates       | 20294     |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 99        |
|    ep_rew_mean     | -99       |
| time/              |           |
|    episodes        | 10        |
|    fps             | 17        |
|    time_elapsed    | 55        |
|    total_timesteps | 990       |
| train/             |           |
|    actor_loss      | -2.51e+04 |
|    

load_replay_buffer
num_trajectories: 100 success rate: 0.9 Reward: 131.0
num_trajectories: 200 success rate: 0.885 Reward: 84.0
num_trajectories: 300 success rate: 0.9 Reward: 76.0
num_trajectories: 400 success rate: 0.9025 Reward: 73.0
num_trajectories: 500 success rate: 0.908 Reward: 76.0
num_trajectories: 600 success rate: 0.9133333333333333 Reward: 81.0
num_trajectories: 700 success rate: 0.9114285714285715 Reward: -35.0
num_trajectories: 800 success rate: 0.91625 Reward: -35.0
num_trajectories: 900 success rate: 0.9177777777777778 Reward: 82.0
num_trajectories: 1000 success rate: 0.917 Reward: 84.0
num_trajectories: 1100 success rate: 0.9163636363636364 Reward: 80.0
num_trajectories: 1200 success rate: 0.9166666666666666 Reward: 82.0
num_trajectories: 1300 success rate: 0.9192307692307692 Reward: 129.0
num_trajectories: 1400 success rate: 0.9171428571428571 Reward: 76.0
num_trajectories: 1500 success rate: 0.9186666666666666 Reward: 90.0
num_trajectories: 1600 success rate: 0.9168

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99.2     |
|    ep_rew_mean     | 182      |
| time/              |          |
|    episodes        | 4850     |
|    fps             | 14       |
|    time_elapsed    | 11       |
|    total_timesteps | 480173   |
| train/             |          |
|    actor_loss      | -50      |
|    critic_loss     | 1.89     |
|    ent_coef        | 0.005    |
|    ent_coef_loss   | 33.6     |
|    learning_rate   | 0.001    |
|    n_updates       | 499972   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99.2     |
|    ep_rew_mean     | 167      |
| time/              |          |
|    episodes        | 4855     |
|    fps             | 15       |
|    time_elapsed    | 43       |
|    total_timesteps | 480668   |
| train/             |          |
|    actor_loss      | -58.2    |
|    critic_loss     | 1.52     |
|    ent_coef 

finish learning


In [ ]:
import gymnasium as gym
import numpy as np
import roboverse

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3 import DDPG, HerReplayBuffer
from sb3_contrib import TQC
from sb3_contrib.common.wrappers import TimeFeatureWrapper
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


env = roboverse.make("Widow250PickPlace-v2",
                         gui=False,
                         observation_mode="pixels",
                         transpose_image=False)
#env = TimeFeatureWrapper(env)
#env = DummyVecEnv([make_env("Widow250PickPlace-v1", i) for i in range(4)])
seed = 1
obs = env.reset()

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path=f"./data/seed_{seed}/",
  name_prefix="tqc_model",
  save_replay_buffer=False,
  save_vecnormalize=False,
)

# model = TQC(env=env, batch_size=2048, buffer_size=1_000_000, gamma=0.95, learning_rate=0.001, policy='MultiInputPolicy',
#              policy_kwargs=dict(net_arch=[512, 512, 512], n_critics=2),
#              replay_buffer_class=HerReplayBuffer,
#              replay_buffer_kwargs=dict(goal_selection_strategy='future', n_sampled_goal=4),
#              tau=0.05, learning_starts=200, verbose=1)

model = TQC.load(f"data/seed_{seed}/tqc_pick_place", env=env)
model.set_env(env)
print("load_replay_buffer")
model.load_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")

# print("start pre-training from buffer only")
# model.learn(total_timesteps=0, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
# model.train(gradient_steps=20000)

print("start learning")
model.learn(total_timesteps=480_000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save(f"data/seed_{seed}/tqc_pick_place")

print("load_replay_buffer")
model.load_replay_buffer(f"data/seed_{seed}/tqc_expert_pick_place")

model.learn(total_timesteps=500_000, callback=checkpoint_callback, log_interval=5, tb_log_name="exp", reset_num_timesteps = False, progress_bar=True)
model.save(f"data/seed_{seed}/tqc_pick_place")
print("finish learning")

/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
load_replay_buffer


/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/stable_baselines3/her/her_replay_buffer.py:394: UserWarning: The last trajectory in the replay buffer will be truncated.
If you are in the same episode as when the replay buffer was saved,
you should use `truncate_last_trajectory=False` to avoid that issue.
  warnings.warn(


...

/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: 
WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: int64
  logger.warn(

/raid/notebook/enhupgu/cog/cogv/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: 
env.compute_reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this 
variable you can do `env.unwrapped.compute_reward` for environment variables or 
`env.get_wrapper_attr('compute_reward')` that will search the reminding wrappers.
  logger.warn(

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | 199      |
| time/              |          |
|    episodes        | 9900     |
|    fps             | 9        |
|    time_elapsed    | 21       |
|    total_timesteps | 980198   |
| train/             |          |
|    actor_loss      | -42.1    |
|    critic_loss     | 0.99     |
|    ent_coef        | 0.00431  |
|    ent_coef_loss   | 19.2     |
|    learning_rate   | 0.001    |
|    n_updates       | 999997   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | 195      |
| time/              |          |
|    episodes        | 9905     |
|    fps             | 9        |
|    time_elapsed    | 73       |
|    total_timesteps | 980693   |
| train/             |          |
|    actor_loss      | -129     |
|    critic_loss     | 8.1      |
|    ent_coef 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



----------------------------------
| rollout/           |           |
|    ep_len_mean     | 99        |
|    ep_rew_mean     | -98.5     |
| time/              |           |
|    episodes        | 10255     |
|    fps             | 9         |
|    time_elapsed    | 3672      |
|    total_timesteps | 1015343   |
| train/             |           |
|    actor_loss      | -1.94e+09 |
|    critic_loss     | 5.25e+07  |
|    ent_coef        | 3.44e+05  |
|    ent_coef_loss   | 0.0442    |
|    learning_rate   | 0.001     |
|    n_updates       | 1035142   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 99        |
|    ep_rew_mean     | -98.5     |
| time/              |           |
|    episodes        | 10260     |
|    fps             | 9         |
|    time_elapsed    | 3725      |
|    total_timesteps | 1015838   |
| train/             |           |
|    actor_loss      | -2.03e+09 |
|    critic_loss    